# Exploratory Data Analysis

Notebook para conter a EDA dos dados do projeto

In [1]:
import pandas as pd
import numpy as np

import pickle as pkl

import seaborn as sns
from matplotlib import pyplot as plt

from joblib import Parallel, delayed
from pathlib import Path

In [13]:
import gc
import sys
import warnings
from joblib import Parallel, delayed
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

warnings.simplefilter("ignore")


2023-02-26 22:42:09.335366: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 22:42:10.014197: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-26 22:42:10.014223: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-26 22:42:11.679928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# File paths for working locally
raw_data_path = '../data/raw-data/'
processed_data_path = '../data/processed-data/'

In [3]:
dataset_names = {
    'Awards': 'awards.csv', 
    'Example': 'example_test.csv', 
    'Players': 'players.csv',
    'Seasons': 'seasons.csv', 
    'Teams': 'teams.csv', 
    'Train': 'train.csv'
}
for key in dataset_names:
  dataset_names[key] = raw_data_path + dataset_names[key]
dataset_names

{'Awards': '../data/raw-data/awards.csv',
 'Example': '../data/raw-data/example_test.csv',
 'Players': '../data/raw-data/players.csv',
 'Seasons': '../data/raw-data/seasons.csv',
 'Teams': '../data/raw-data/teams.csv',
 'Train': '../data/raw-data/train.csv'}

In [4]:
# training = pd.read_csv(dataset_names['Train'])
# pd.to_pickle(df, processed_data_path + 'train.pkl')

In [5]:
training = pd.read_pickle(processed_data_path + 'train.pkl')

In [6]:
players = pd.read_csv(dataset_names['Players'])

In [7]:
training.info()
training.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   date                     1216 non-null   int64 
 1   nextDayPlayerEngagement  1216 non-null   object
 2   games                    639 non-null    object
 3   rosters                  1216 non-null   object
 4   playerBoxScores          538 non-null    object
 5   teamBoxScores            538 non-null    object
 6   transactions             1103 non-null   object
 7   standings                531 non-null    object
 8   awards                   294 non-null    object
 9   events                   536 non-null    object
 10  playerTwitterFollowers   40 non-null     object
 11  teamTwitterFollowers     40 non-null     object
dtypes: int64(1), object(11)
memory usage: 114.1+ KB


,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [8]:
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return pd.DataFrame() if pd.isna(json_str) else pd.read_json(json_str)


def unpack_data(data, dfs=None, n_jobs=-1):
    if dfs is not None:
        data = data.loc[:, dfs]
    unnested_dfs = {}
    for name, column in data.iteritems():
        daily_dfs = Parallel(n_jobs=n_jobs)(
            delayed(unpack_json)(item) for date, item in column.iteritems())
        df = pd.concat(daily_dfs)
        unnested_dfs[name] = df
    return unnested_dfs

In [9]:
%%time
# Define dataframes to load from training set
dfs = [
    'nextDayPlayerEngagement',  # targets
    'playerBoxScores',  # features
    # Other dataframes available for features:
    # 'games',
    # 'rosters',
    # 'teamBoxScores',
    # 'transactions',
    # 'standings',
    # 'awards',
    # 'events',
    # 'playerTwitterFollowers',
    # 'teamTwitterFollowers',
]

# Convert training data date field to datetime type
training['date'] = pd.to_datetime(training['date'], format="%Y%m%d")
training = training.set_index('date').to_period('D')
print(training.info())

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 1216 entries, 2018-01-01 to 2021-04-30
Freq: D
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   nextDayPlayerEngagement  1216 non-null   object
 1   games                    639 non-null    object
 2   rosters                  1216 non-null   object
 3   playerBoxScores          538 non-null    object
 4   teamBoxScores            538 non-null    object
 5   transactions             1103 non-null   object
 6   standings                531 non-null    object
 7   awards                   294 non-null    object
 8   events                   536 non-null    object
 9   playerTwitterFollowers   40 non-null     object
 10  teamTwitterFollowers     40 non-null     object
dtypes: object(11)
memory usage: 114.0+ KB
None
CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 14.5 ms


In [10]:
%time
# Unpack nested dataframes and store in dictionary `training_dfs`
training_dfs = unpack_data(training, dfs=dfs)
print('\n', training_dfs.keys())

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.44 µs

 dict_keys(['nextDayPlayerEngagement', 'playerBoxScores'])


In [11]:
# Players in the test set. We'll filter our data for only this set of players
pids_test = players.playerId.loc[
    players.playerForTestSetAndFuturePreds.fillna(False)
].astype(str)

# Name of target columns
targets = ["target1", "target2", "target3", "target4"]


def make_playerBoxScores(dfs: dict, features):
    X = dfs['playerBoxScores'].copy()
    X = X[['gameDate', 'playerId'] + features]
    # Set dtypes
    X = X.astype({name: np.float32 for name in features})
    X = X.astype({'playerId': str})
    # Create date index
    X = X.rename(columns={'gameDate': 'date'})
    X['date'] = pd.PeriodIndex(X.date, freq='D')
    # Aggregate multiple games per day by summing
    X = X.groupby(['date', 'playerId'], as_index=False).sum()
    return X


def make_targets(training_dfs: dict):
    Y = training_dfs['nextDayPlayerEngagement'].copy()
    # Set dtypes
    Y = Y.astype({name: np.float32 for name in targets})
    Y = Y.astype({'playerId': str})
    # Match target dates to feature dates and create date index
    Y = Y.rename(columns={'engagementMetricsDate': 'date'})
    Y['date'] = pd.to_datetime(Y['date'])
    Y = Y.set_index('date').to_period('D')
    Y.index = Y.index - 1
    return Y.reset_index()


def join_datasets(dfs):
    dfs = [x.pivot(index='date', columns='playerId') for x in dfs]
    df = pd.concat(dfs, axis=1).stack().reset_index('playerId')
    return df


def make_training_data(training_dfs: dict,
                       features,
                       targets,
                       fourier=4,
                       test_size=30):
    # Process dataframes
    X = make_playerBoxScores(training_dfs, features)
    Y = make_targets(training_dfs)
    # Merge for processing
    df = join_datasets([X, Y])
    # Filter for players in test set
    df = df.loc[df.playerId.isin(pids_test), :]
    # Convert from long to wide format
    df = df.pivot(columns="playerId")
    # Restore features and targets
    X = df.loc(axis=1)[features, :]
    Y = df.loc(axis=1)[targets, :]
    # Fill missing values in features
    X.fillna(-1, inplace=True)
    # Create temporal features
    fourier_terms = CalendarFourier(freq='A', order=fourier)
    deterministic = DeterministicProcess(
        index=X.index,
        order=0,
        seasonal=False,  # set to True for weekly seasonality
        additional_terms=[fourier_terms],
    )
    X = pd.concat([X, deterministic.in_sample()], axis=1)
    # Create train / validation splits
    X_train, X_valid, y_train, y_valid = train_test_split(
        X,
        Y,
        test_size=test_size,
        shuffle=False,
    )
    return X_train, X_valid, y_train, y_valid, deterministic

In [18]:
%%time
# Columns to select from playerBoxScores, all numeric
features = [
    "hits",
    "strikeOuts",
    "homeRuns",
    "runsScored",
    "stolenBases",
    "strikeOutsPitching",
    "inningsPitched",
    "strikes",
    "flyOuts",
    "groundOuts",
    "errors",
]

# Number of days to use for the validation set
test_size = 1

X_train, X_valid, y_train, y_valid, deterministic = make_training_data(
    training_dfs, 
    features=features, 
    targets=targets,
    fourier=4,  # number of Fourier pairs describing annual seasonality
    test_size=test_size,
)

CPU times: user 16.1 s, sys: 2.41 s, total: 18.5 s
Wall time: 18.7 s


In [19]:
y_train

target1                                                      \
playerId       405395     408234    424144    425772    425784     425794   
date                                                                        
2018-01-01   0.151872   0.029593  0.001117  0.023451  0.002233   2.343410   
2018-01-02   0.136406   0.028846  0.000489  0.012223  0.000978   0.594516   
2018-01-03   0.064460   0.017782  0.000889  0.008447  0.006224   0.249395   
2018-01-04   0.024217   0.015472  0.000000  0.005045  0.001682   0.121756   
2018-01-05   0.157556   0.070215  0.000000  0.015413  0.011988   0.318537   
...               ...        ...       ...       ...       ...        ...   
2021-04-25  12.577413  16.977015  0.001707  0.000569  0.000569   2.720312   
2021-04-26  21.337193  16.311810  0.001622  0.001460  0.000000  13.710207   
2021-04-27   9.151484  10.927429  0.338001  0.001498  0.000346   0.257475   
2021-04-28   0.890671  10.598249  0.067611  0.001046  0.000000   0.538561   
2021-04-29   0.962072   0.942823  0.018531  0.000837  0.000359   3.286013   

                                                    ... target4            \
playerId      425844    425877    429722    431148  ...  680430    680692   
date                                                ...                     
2018-01-01  0.006700  0.217758  0.011725  0.000000  ...     0.0  0.000000   
2018-01-02  0.009778  0.136406  0.002445  0.000489  ...     0.0  0.000000   
2018-01-03  0.005779  0.097802  0.011114  0.002223  ...     0.0  0.000000   
2018-01-04  0.003363  0.051797  0.013790  0.000673  ...     0.0  0.000000   
2018-01-05  0.015413  0.207220  0.029114  0.000000  ...     0.0  0.000000   
...              ...       ...       ...       ...  ...     ...       ...   
2021-04-25  0.112825  1.558879  0.000758  0.002275  ...     0.0  0.288392   
2021-04-26  0.096860  2.252613  0.001298  0.001136  ...     0.0  0.113972   
2021-04-27  0.414840  0.229020  0.000461  0.000922  ...     0.0  0.152672   
2021-04-28  0.121629  0.110245  0.000232  0.001046  ...     0.0  0.103220   
2021-04-29  0.042084  0.076755  0.000120  0.000956  ...     0.0  0.143248   

                                                                        \
playerId      680704    680777    680911    680963    681911    683232   
date                                                                     
2018-01-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-01-02  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-01-03  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-01-04  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2018-01-05  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...              ...       ...       ...       ...       ...       ...   
2021-04-25  0.058989  0.478469  0.144196  0.013109  0.111424  0.052435   
2021-04-26  0.026817  0.174309  0.040225  0.026817  0.093859  0.013408   
2021-04-27  0.610687  0.161652  0.062865  0.080826  0.242479  0.098788   
2021-04-28  1.321222  0.185797  0.020644  0.041288  0.309661  0.072254   
2021-04-29  1.276208  0.586014  0.078135  0.078135  0.299518  0.130225   

                                
playerId      683734    685503  
date                            
2018-01-01  0.000000  0.098039  
2018-01-02  0.000000  0.240602  
2018-01-03  0.000000  0.085985  
2018-01-04  0.000000  0.184162  
2018-01-05  0.000000  0.021363  
...              ...       ...  
2021-04-25  0.734089  1.055253  
2021-04-26  0.858139  0.737463  
2021-04-27  1.230355  0.790301  
2021-04-28  2.477292  0.970272  
2021-04-29  2.122672  5.938273  

[1215 rows x 4748 columns]

In [20]:
training

,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
date,,,,,,,,,,,
2018-01-01,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
2018-01-02,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2018-01-03,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
2018-01-04,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
2018-01-05,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-26,"[{""engagementMetricsDate"":""2021-04-27"",""player...","[{""gamePk"":634374,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-26"",""t...","[{""home"":1,""gamePk"":634377,""gameDate"":""2021-04...","[{""home"":1,""teamId"":139,""gamePk"":634343,""gameD...","[{""transactionId"":480386,""playerId"":543685,""pl...","[{""season"":2021,""gameDate"":""2021-04-26"",""divis...",NaN,"[{""gamePk"":634433,""gameDate"":""2021-04-26"",""gam...",NaN,NaN
2021-04-27,"[{""engagementMetricsDate"":""2021-04-28"",""player...","[{""gamePk"":634318,""gameType"":""R"",""season"":2021...","[{""playerId"":443558,""gameDate"":""2021-04-27"",""t...","[{""home"":1,""gamePk"":634320,""gameDate"":""2021-04...","[{""home"":1,""teamId"":117,""gamePk"":634333,""gameD...","[{""transactionId"":480456,""playerId"":642162,""pl...","[{""season"":2021,""gameDate"":""2021-04-27"",""divis...",NaN,"[{""gamePk"":634332,""gameDate"":""2021-04-27"",""gam...",NaN,NaN
2021-04-28,"[{""engagementMetricsDate"":""2021-04-29"",""player...","[{""gamePk"":634309,""gameType"":""R"",""season"":2021...","[{""playerId"":429722,""gameDate"":""2021-04-28"",""t...","[{""home"":1,""gamePk"":634310,""gameDate"":""2021-04...","[{""home"":0,""teamId"":111,""gamePk"":634310,""gameD...","[{""transactionId"":480728,""playerId"":545358,""pl...","[{""season"":2021,""gameDate"":""2021-04-28"",""divis...",NaN,"[{""gamePk"":634317,""gameDate"":""2021-04-28"",""gam...",NaN,NaN
